In [ ]:
#=============================================================================
# Solar Farm Semantic Segmentation with U-Net - Reed Johnson - Acreage Calculation
# -----------------------------------------------------------------------------

# -------------------- Import Libraries --------------------
import os
from PIL import Image
import numpy as np

In [ ]:
# -------------------- Set Paths and Parameters --------------------
mask_dir = "/workspace/data_to_runpod/masks_1024ft_v2"
pred_dir = "/workspace/outputs"

tile_size_ft = 1024
image_dim_px = 512
pixel_area_sqft = (tile_size_ft / image_dim_px) ** 2

In [ ]:
# -------------------- Mask Loading Function --------------------
# Converts grayscale image to binary mask: 1 where pixel == 255, else 0
def load_binary_mask(path):
    mask = np.array(Image.open(path).convert("L"))
    return (mask == 255).astype(np.uint8)

In [ ]:
# -------------------- Collect Matched Mask Files --------------------
# Only include masks with corresponding predictions
mask_files = sorted([
    f for f in os.listdir(mask_dir)
    if f.endswith(".png") and os.path.exists(os.path.join(pred_dir, f.replace("mask_", "tile_").replace(".png", "_pred.png")))
])

In [ ]:
# ----------------------- Pixel Counts -----------------------
total_mask_pixels = 0
total_pred_pixels = 0

for fname in mask_files:
    mask_path = os.path.join(mask_dir, fname)
    pred_path = os.path.join(pred_dir, fname.replace("mask_", "tile_").replace(".png", "_pred.png"))

    gt_mask = load_binary_mask(mask_path)  # Ground truth
    pred_mask = load_binary_mask(pred_path)  # Prediction

    total_mask_pixels += np.sum(gt_mask)  # Count pixels with value 1
    total_pred_pixels += np.sum(pred_mask)

In [ ]:
# -------------------- Area Conversion --------------------
total_mask_area_sqft = total_mask_pixels * pixel_area_sqft
total_pred_area_sqft = total_pred_pixels * pixel_area_sqft

sqft_per_acre = 43560 #sqft per acre
total_mask_area_acres = total_mask_area_sqft / sqft_per_acre
total_pred_area_acres = total_pred_area_sqft / sqft_per_acre

In [ ]:
# -------------------- Print Output --------------------
print(f"Ground Truth Mask Area: {total_mask_area_sqft:,.0f} sq ft ({total_mask_area_acres:.2f} acres)")
print(f"Predicted Mask Area: {total_pred_area_sqft:,.0f} sq ft ({total_pred_area_acres:.2f} acres)")

Ground Truth Mask Area: 7,009,248 sq ft (160.91 acres)
Predicted Mask Area: 6,590,044 sq ft (151.29 acres)
